In [71]:
import numpy as np
import pandas as pd
from scipy.stats import beta, norm, cauchy, gamma


In [89]:

def prior(n, hyper):
    X_1 = beta.rvs(hyper[0], hyper[1] + hyper[2], size=n)
    Temp = beta.rvs(hyper[1], hyper[2], size=n)
    X_2 = Temp * (1 - X_1)
    R_1 = X_1 - X_2
    R_2 = 2 * (X_1 + X_2) - 1
    return np.column_stack((R_1, R_2))

def simu(tht1, tht2, sigm, n):
    A = np.random.normal(0, sigm, size=n + 2)
    U = A[2:(n + 2)]
    V = A[1:(n + 1)]
    W = A[0:n]
    return U + tht1 * V + tht2 * W

def distautcor(Z, X):
    p = len(Z)
    a2 = np.sum(Z[2:p] * Z[0:(p - 2)])
    b2 = np.sum(X[2:p] * X[0:(p - 2)])
    a1 = np.sum(Z[1:p] * Z[0:(p - 1)])
    b1 = np.sum(X[1:p] * X[0:(p - 1)])
    return np.sqrt((a2 - b2)**2 + (a1 - b1)**2)


In [104]:
def ABCbas(X, N, sigm, hyper):
    res = prior(N, hyper)
    dist = np.apply_along_axis(lambda x: distautcor(X - np.mean(X), simu(x[0], x[1], sigm, len(X))), 1, res)
    return res[np.argmin(dist), :]

def statdir(U):
    q = (U[:, 1] + 2 * U[:, 0] + 1) / 4
    q = np.where(q != 0, q, np.finfo(float).eps)  # Remplacer les zéros par une très petite valeur
    
    return np.array([np.sum(np.log(q)), np.sum(np.log(np.abs(q - U[:, 0])))])

    
def ABChaut(Q1, Q2, L):
    Q = np.column_stack((Q1, Q2))
    hyper = np.random.exponential(size=(L, 3))
    dist = np.apply_along_axis(lambda x: np.sum((statdir(prior(len(Q1), x)) - statdir(Q))**2), 1, hyper)
    return hyper[np.argmin(dist), :]

def ABCsigm(Dat, j, parh1, parh2, parsigm1, parsigm2, N):
    dist = np.zeros(N)
    n = Dat.shape[0]
    w = np.arange(0, n, 3)
    si = 1 / gamma.rvs(parsigm1, scale=parsigm2, size=N)
    for i in range(N):
        dist[i] = np.abs(np.var(simu(parh1[j], parh2[j], si[i], n)[w]) - np.var(Dat[w, j]))
    return si[np.argmin(dist)]

def ABCsigmhaut(sigm, Q):
    dist = np.zeros(Q)
    n = len(sigm)
    pri = np.abs(cauchy.rvs(size=2*Q)).reshape((Q, 2))
    for i in range(Q):
        h = pri[i]
        
        # Générer une valeur pour h[0] et h[1]
        while True:
            try:
                y = 1 / gamma.rvs(h[0], scale=h[1], size=n)
                break
            except OverflowError:
                # Si une erreur de dépassement de capacité se produit, ajustez les paramètres h[0] et h[1]
                h = np.abs(cauchy.rvs(size=2))
        
        dist[i] = np.abs(np.sum(np.log(y)) - np.sum(np.log(sigm))) + np.abs(np.sum(y) - np.sum(sigm))
    return pri[np.argmin(dist), :]


# Simule une chaîne selon ABCGibbs,
# renvoie une liste de matrices : hyperparamètres, premier paramètre, second, sigma, hypersigma
# pour les paramètres et sigma, les colonnes correspondent aux séries temporelles.

X: C'est le dataset, les données sur lesquelles nous effectuons les analyses et les simulations.
N: Nombre de simulations ou d'échantillons à générer dans les fonctions ABC pour déduire chaque paramètre
# M simulations pour les hyperparamètres, P simulations pour sigma
# P2 simulations pour l'hypersigma, Npts nombre de points désirés.
sigm: C'est la variance utilisée dans la simulation du modèle AR.
hyper: Les paramètres hyperparamétriques du modèle.
res: Les résultats de la fonction.
dist: Les distances calculées entre les simulations et les données réelles.
U: Les données générées par la fonction prior, utilisées dans les étapes de calcul.
q: La quantité calculée pour les statistiques suffisantes dans la fonction statdir.
Q1 et Q2: Vecteurs des premier et deuxième paramètres respectivement, utilisés dans la fonction ABChaut.
L: Nombre de simulations pour l'inférence des hyperparamètres dans ABChaut.
Dat: L'ensemble de données complet utilisé dans ABCsigm.
j: L'index de la variance à inférer dans ABCsigm.
parsigm1 et parsigm2: Les hyperparamètres sur la variance sigma dans ABCsigm.
Npts: Le nombre de points souhaités dans la fonction Gibbs.

In [91]:

def Gibbs(X, N, M, P, P2, Npts):
    hyper = np.zeros((Npts, 3))
    hyper[0, :] = np.random.exponential(size=3)
    par1 = np.full((Npts, X.shape[1]), np.nan)
    par2 = np.full((Npts, X.shape[1]), np.nan)
    sigm = np.full((Npts, X.shape[1]), np.nan)
    hypersigm = np.full((Npts, 2), np.nan)
    hypersigm[0, :] = np.abs(cauchy.rvs(size=2))
    sigm[0, :] = 1 / gamma.rvs(hypersigm[0, 0], scale=hypersigm[0, 1], size=X.shape[1])
    d = prior(X.shape[1], hyper[0, :])
    par1[0, :] = d[:, 0]
    par2[0, :] = d[:, 1]
    m = X.shape[1]
    V = X - np.tile(np.mean(X, axis=0), (X.shape[0], 1))
    for i in range(1, Npts):
        for j in range(X.shape[1]):
            U = ABCbas(X[:, j], N, sigm[i - 1, :], hyper[i - 1, :])
            par1[i, j] = U[0]
            par2[i, j] = U[1]
            sigm[i, j] = ABCsigm(X, j, par1[i, :], par2[i, :], hypersigm[i - 1, 0], hypersigm[i - 1, 1], P)
        hyper[i, :] = ABChaut(par1[i, :], par2[i, :], M)
        hypersigm[i, :] = ABCsigmhaut(sigm[i, :], P2)
    return [hyper, par1, par2, sigm, hypersigm]


In [93]:

def Gibbs(X, N, M, P, P2, Npts):
    hyper = np.zeros((Npts, 3))
    hyper[0, :] = np.random.exponential(size=3)
    par1 = np.full((Npts, X.shape[1]), np.nan)
    par2 = np.full((Npts, X.shape[1]), np.nan)
    sigm = np.full((Npts, X.shape[1]), np.nan)
    hypersigm = np.full((Npts, 2), np.nan)
    hypersigm[0, :] = np.abs(cauchy.rvs(size=2))
    sigm[0, :] = 1 / gamma.rvs(hypersigm[0, 0], scale=hypersigm[0, 1], size=X.shape[1])
    d = prior(X.shape[1], hyper[0, :])
    par1[0, :] = d[:, 0]
    par2[0, :] = d[:, 1]
    m = X.shape[1]
    V = X - np.tile(np.mean(X, axis=0), (X.shape[0], 1))
    for i in range(1, Npts):
        for j in range(X.shape[1]):
            U = ABCbas(X[:, j], N, sigm[i - 1, :], hyper[i - 1, :])
            par1[i, j] = U[0]
            par2[i, j] = U[1]
            sigm[i, j] = ABCsigm(X, j, par1[i, :], par2[i, :], hypersigm[i - 1, 0], hypersigm[i - 1, 1], P)
        hyper[i, :] = ABChaut(par1[i, :], par2[i, :], M)
        hypersigm[i, :] = ABCsigmhaut(sigm[i, :], P2)
    return [hyper, par1, par2, sigm, hypersigm]


In [ ]:
l=50
X = np.random.normal(-5, 1, (48, l)) #data

In [109]:
toyG = Gibbs(X, 100, 10, 10, 10, 100)
toyG

C:\Users\sarah\AppData\Local\Temp\ipykernel_36972\1030311358.py:10: RuntimeWarning: divide by zero encountered in log
  return np.array([np.sum(np.log(q)), np.sum(np.log(np.abs(q - U[:, 0])))])
C:\Users\sarah\AppData\Local\Temp\ipykernel_36972\1030311358.py:38: RuntimeWarning: divide by zero encountered in divide
  y = 1 / gamma.rvs(h[0], scale=h[1], size=n)


[array([[1.17043508, 0.9559411 , 1.41463267],
        [1.64276396, 1.32248403, 0.38752358],
        [2.65071208, 2.13492511, 1.11042617],
        [1.29089686, 0.71429336, 0.10126577],
        [1.02752778, 1.03063728, 0.40087673],
        [1.16564324, 1.14751697, 2.29403591],
        [2.05516822, 2.79485841, 1.74275832],
        [1.95276584, 2.60643626, 1.68885868],
        [0.67573621, 1.32900866, 0.14505142],
        [0.94185688, 0.93837869, 0.50570323],
        [2.3417261 , 1.77706092, 2.05775404],
        [1.43308724, 2.26820033, 0.89197131],
        [1.94276312, 3.5625691 , 0.12939366],
        [2.71954983, 2.91535287, 0.04461543],
        [3.6505153 , 4.33809624, 2.82805037],
        [0.95954159, 0.8961481 , 0.22669471],
        [0.66145762, 1.3052368 , 0.45631609],
        [0.69369166, 1.43127502, 2.37424733],
        [0.69343311, 1.75637395, 1.12148751],
        [0.62878867, 0.67663053, 1.0720389 ],
        [0.88118969, 0.57794838, 0.85469035],
        [0.75427831, 0.64595068, 1

In [97]:
l=50
X = np.random.normal(-5, 1, (50, l)) #data

N=1000
M=100
P=100
P2=100
Npts=1000

hyper = np.zeros((Npts, 3))
hyper[0, :] = np.random.exponential(size=3)
par1 = np.full((Npts, X.shape[1]), np.nan)
par2 = np.full((Npts, X.shape[1]), np.nan)
sigm = np.full((Npts, X.shape[1]), np.nan)
hypersigm = np.full((Npts, 2), np.nan)
hypersigm[0, :] = np.abs(cauchy.rvs(size=2))
sigm[0, :] = 1 / gamma.rvs(hypersigm[0, 0], scale=hypersigm[0, 1], size=X.shape[1])
d = prior(X.shape[1], hyper[0, :])
par1[0, :] = d[:, 0]
par2[0, :] = d[:, 1]
m = X.shape[1]
V = X - np.tile(np.mean(X, axis=0), (X.shape[0], 1))

print(sigm.shape)
print(X.shape)

(1000, 50)
(50, 50)


In [98]:
X=pd.read_csv('toydata.csv', delim_whitespace=True)
X = np.array(X)

C:\Users\sarah\AppData\Local\Temp\ipykernel_36972\3137123313.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X=pd.read_csv('toydata.csv', delim_whitespace=True)
